In [2]:
import attr

from quantus_nlp.xai import LitLimeModelAdapter, LitDatasetAdapter
from lit_nlp.components import lime_explainer, gradient_maps
import pandas as pd

In [3]:
ds = LitDatasetAdapter()

nn = LitLimeModelAdapter()

## Lime

In [4]:
lime = lime_explainer.LIME()

lime_results = lime.run(ds.examples[:1], nn, ds)[0]

{'text': TokenSalience(tokens=['CHARLOTTE,', 'N.C.', '(Sports', 'Network)', '-', 'Carolina', 'Panthers', 'running', 'back', 'Stephen', 'Davis', 'will', 'miss', 'the', 'remainder', 'of', 'the', 'season', 'after', 'being', 'placed', 'on', 'injured', 'reserve', 'Saturday.'], salience=array([ 0.04327561,  0.02655925,  0.0999539 ,  0.01275676,  0.05141004,
         0.06659845,  0.1627766 ,  0.03524831,  0.01917952,  0.01403224,
         0.0357442 ,  0.03952775,  0.02323452,  0.01087326, -0.01781326,
         0.01997427,  0.04865328,  0.04661352,  0.0418358 ,  0.01939604,
         0.01455986, -0.01218454,  0.04352131,  0.06039166,  0.03388604]))}

In [10]:
pd.DataFrame(attr.asdict(lime_results['text'])).sort_values(by='salience', ascending=False)

,tokens,salience
6,Panthers,0.162777
2,(Sports,0.099954
5,Carolina,0.066598
23,reserve,0.060392
4,-,0.051410
16,the,0.048653
17,season,0.046614
22,injured,0.043521
0,"CHARLOTTE,",0.043276
18,after,0.041836
